In [37]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from collections import Counter
import pandas as pd

# Ensure NLTK punkt tokenizer models are downloaded
nltk.download('punkt', download_dir='/Users/moiz/nltk_data')
nltk.data.path.append('/Users/moiz/nltk_data')

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

def calculate_bleu(reference, candidate):
    reference_tokens = [nltk.word_tokenize(reference)]
    candidate_tokens = nltk.word_tokenize(candidate)
    smooth = SmoothingFunction().method1
    return sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smooth)

def calculate_meteor(reference, candidate):
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)
    return meteor_score([reference_tokens], candidate_tokens)

def calculate_rouge(reference, candidate):
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)[0]
    return scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']

def calculate_f1(reference, candidate):
    reference_tokens = nltk.word_tokenize(reference)
    candidate_tokens = nltk.word_tokenize(candidate)
    common = Counter(reference_tokens) & Counter(candidate_tokens)
    true_positives = sum(common.values())
    pred_positives = len(candidate_tokens)
    real_positives = len(reference_tokens)
    precision = true_positives / pred_positives if pred_positives > 0 else 0
    recall = true_positives / real_positives if real_positives > 0 else 0
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

[nltk_data] Downloading package punkt to /Users/moiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Main Data

In [38]:
path_to_referenceMain = 'referenceMain.txt' 

reference_textMain = read_file(path_to_referenceMain)

candidate_textMain_Mistral = "Daraz's refund policy allows for returns or refunds on items that are damaged, defective, incorrect, or incomplete. However, for device-related issues after usage or beyond the return policy period, it is recommended to check if the item is covered under Seller or Brand Warranty. Items that are non-returnable include any custom-made items. For Buy More, Save More orders, partial cancellations are not permitted, but partial returns are allowed. This means that if you wish to return certain items from a Buy More, Save More order, you can do so, but you would need to cancel the entire order if you want to return one or two specific products. After the returned items have been quality checked by Daraz, a partial refund will be issued to the customer on an item-level basis for each bundle. To submit a return request, customers must do so within 14 days of receiving their order. Decisions on returns and refunds will be taken by Daraz in its sole discretion. Customers attempting to circumvent the terms and conditions and abuse subsidies by creating new accounts will be unable to place their orders."
candidate_textMain_Phi = "Thank you for your email. We appreciate your inquiry. Daraz has a clear cancellation and return policy for Buy More, Save More orders. Full refunds are only provided for damaged, defective, incorrect, or incomplete items. If the item is a device-related issue after the return policy period or not covered under the Seller or Brand Warranty, the refund may not be provided. Regarding partial cancellations, Daraz does not allow it. However, customers have the option to return certain items out of their total Buy More, Save More order. If a certain number of products from the order are eligible for a return, a partial refund will be issued after the items have been returned by the customer and checked by Daraz. In case you are not satisfied with your purchase, you can submit a return request within 14 days from the delivery date. Daraz makes the final decision on returns and refunds, and they will be provided in their sole discretion. Customers are not allowed to circumvent the terms and conditions and abuse subsidies by creating new accounts."
candidate_textMain_Qwen = "Daraz offers a refund policy for any damaged, defective, incorrect, or incomplete items received. The policy states that for device-related issues after usage or expiration of return policy period, the customer may need to check if the item is covered under seller or brand warranty before returning it. Partial cancellations are not allowed, but partial returns are allowed on Buy More, Save More orders, where the customer can return certain items out of their total order and receive a partial refund on an item-level basis. If a certain number of products from a Buy More, Save More order are eligible for a return, partial refund will be issued to the customer after these items have been returned by the customer and quality checked by Daraz. Partial refunds are calculated on item level of each bundle. In case of dissatisfaction with the purchase, customers can submit a return request within 14 days from the day it is delivered to them. Daraz will take a decision on returns and refunds based on its sole discretion. Customers who try to circumvent the terms and conditions and abuse subsidies by creating new accounts will be unable to place their order. Overall, Daraz's refund policy provides customers with peace of mind when dealing with damaged, defective, incorrect, or incomplete items. Customers can request partial refunds on certain items out of their total order and claim partial refunds on an item-level basis. The policy also allows for partial cancellations, which means that customers can cancel up to two products out of their total order without having to cancel the entire order."
candidate_textMain_Zephyr = "If you receive a damaged, defective, incorrect, or incomplete item, Daraz will assess the situation and issue a refund based on their evaluation. However, for device-related issues that arise after usage or beyond the return policy period, we suggest checking if the item is covered under the seller or brand warranty. Items that are custom-made cannot be returned. In terms of the cancellation and return policy for Buy More, Save More orders, partial cancellations are not allowed, but partial returns are allowed. This means that if you decide to cancel your order, you cannot cancel just one or two products; instead, you must cancel the entire order. However, if you choose to return certain items from your Buy More, Save More order, a partial refund will be issued to you after these items have been returned and quality checked by Daraz. The partial refund is calculated on an item level for each bundle. If you receive your order but are not satisfied with your purchase, you can submit a return request within 14 days from the date it is delivered to you. Daraz will make the final decision on returns and refunds in their sole discretion. Additionally, we want to remind our customers that attempting to bypass our terms and conditions and abuse subsidies by creating new accounts will prevent you from placing your order. In summary, if you receive a damaged, defective, incorrect, or incomplete item, Daraz will assess the situation and issue a refund based on their evaluation. For device-related issues, we suggest checking the seller or brand warranty. Partial cancellations are not allowed for Buy More, Save More orders, but partial returns are allowed, and a partial refund will be issued on an item level for each bundle. If you are not satisfied with your purchase, you can submit a return request within 14 days of delivery. Our final decision on returns and refunds is at our sole discretion. And lastly, creating new accounts to bypass our terms and conditions and abuse subsidies will not be permitted."

candidate_texts = {
    "Mistral": candidate_textMain_Mistral,
    "Phi": candidate_textMain_Phi,
    "Qwen": candidate_textMain_Qwen,
    "Zephyr": candidate_textMain_Zephyr
}

resultsMain = []

for model, candidate_text in candidate_texts.items():
    bleu = calculate_bleu(reference_textMain, candidate_text)
    meteor = calculate_meteor(reference_textMain, candidate_text)
    rouge_1, rouge_2, rouge_l = calculate_rouge(reference_textMain, candidate_text)
    f1 = calculate_f1(reference_textMain, candidate_text)
    resultsMain.append({
        "Model": model,
        "BLEU": bleu,
        "METEOR": meteor,
        "ROUGE-1": rouge_1,
        "ROUGE-2": rouge_2,
        "ROUGE-L": rouge_l,
        "F1 Score": f1
    })

dfMain = pd.DataFrame(resultsMain)
dfMain

,Model,BLEU,METEOR,ROUGE-1,ROUGE-2,ROUGE-L,F1 Score
0,Mistral,0.065268,0.222387,0.282528,0.097938,0.282528,0.344969
1,Phi,0.051876,0.219387,0.274510,0.068602,0.258824,0.368201
2,Qwen,0.071454,0.244083,0.286713,0.099323,0.279720,0.357895
3,Zephyr,0.052465,0.266486,0.320557,0.096280,0.306620,0.346386


# Seller

In [39]:
path_to_referenceMain = 'referenceMain.txt' 

reference_textMain = read_file(path_to_referenceMain)

candidate_textMain_Mistral = "Absolutely, Daraz allows sellers to create product bundles as a sales promotion strategy. Product bundling is an effective way to differentiate deals from competitors, especially in high-priced categories, and it provides several benefits. Firstly, bundles enable sellers to sell low-cost items together with more expensive ones, offering customers a better value for their money. This can be a great opportunity to clear inventory of certain products. Additionally, bundles are an excellent way to attract customers and increase conversion rates. By selling a package or set of goods for a lower price than the customer would pay if they bought each item separately, sellers can enhance overall performance and boost sales. There are two common types of product bundling: 1. Buy One Get One Free (BOGO): This is a form of sales promotion where the customer receives one product for free when they buy another product. It is not actually half price, but it can be an effective way to generate interest and sales. 2. Combo: This is a promotion where different products are sold together as a set with one or more items at discounted prices. Moreover, if a seller is selling only custom-made products, they should aim to have a minimum of 5 such products in their bundle. As for the benefits of product customization, it can help match customer demand, increase sales and competitive edge, boost customer engagement, build brand loyalty, and gain valuable market research. By offering customized products, sellers can cater to individual customer preferences, which can lead to repeat business and positive word-of-mouth marketing."
candidate_textMain_Phi = "Of course! I'd be happy to guide you through the process of creating a custom bundle on Daraz. Here's what you need to do: 1. Log in to your Daraz account. 2. Go to your \"Products\" tab and select the products you want to include in your bundle. You can choose up to five products per bundle. 3. Once you have selected your products, click on the \"Create Bundle\" button."
candidate_textMain_Qwen = "Daraz offers various types of product bundling options such as minimum 5 customs products required for a bundle sale, combo A promotions, and low-cost item packages. The benefits of product customization include matching customer demand, increasing sales and competitive edge, building brand loyalty, and gaining market research insights. By creating bundles, sellers can differentiate themselves from competitors and attract customers who may be interested in purchasing a set of products at a lower price. Additionally, bundle sales can help sellers clear their inventory and reduce costs associated with promoting individual products. Overall, product bundling can be an effective marketing strategy for sellers looking to increase conversions, sales, and overall performance."
candidate_textMain_Zephyr = "Yes, you can definitely make product bundles on Daraz. Daraz offers a feature called \"Bundles\" which allows sellers to create promotional offers by bundling multiple products together and selling them at a discounted price. This is a great way to increase sales, attract customers, and clear inventory. There are different types of product bundling that you can choose from, such as \"Buy 1 Get 1 Free\" or \"Combo\" bundles.  In \"Buy 1 Get 1 Free\" bundles, you offer a second product for free when the customer buys a certain product. This is a popular promotion that helps you differentiate your deals from competitors, especially in high-priced competitive categories. Customers are attracted to this type of bundle because they get more value for their money. In \"Combo\" bundles, you sell multiple products together as a set at a discounted price. This is a great opportunity to sell low-cost items as part of a bundle. It also helps you move slow-moving inventory by bundling it with more popular products. To create a bundle on Daraz, you need to have a minimum of 5 custom products if you are selling only custom-made products. Customers can customize the products according to their preferences, which can help you match customer demand, increase sales, and gain market research. Creating bundles has several benefits, including increasing sales and competitive edge, customer engagement, brand loyalty, and enhancing overall performance. By creating bundles, you can also attract customers and increase conversion rates. In summary, Daraz's Bundles feature is a great way to offer promotional deals, clear inventory, and attract customers. By following the guidelines and best practices for creating bundles, you can maximize the benefits and improve your overall performance on the platform."

candidate_texts = {
    "Mistral": candidate_textMain_Mistral,
    "Phi": candidate_textMain_Phi,
    "Qwen": candidate_textMain_Qwen,
    "Zephyr": candidate_textMain_Zephyr
}

resultsSeller = []

for model, candidate_text in candidate_texts.items():
    bleu = calculate_bleu(reference_textMain, candidate_text)
    meteor = calculate_meteor(reference_textMain, candidate_text)
    rouge_1, rouge_2, rouge_l = calculate_rouge(reference_textMain, candidate_text)
    f1 = calculate_f1(reference_textMain, candidate_text)
    resultsSeller.append({
        "Model": model,
        "BLEU": bleu,
        "METEOR": meteor,
        "ROUGE-1": rouge_1,
        "ROUGE-2": rouge_2,
        "ROUGE-L": rouge_l,
        "F1 Score": f1
    })

dfSeller = pd.DataFrame(resultsSeller)
dfSeller

,Model,BLEU,METEOR,ROUGE-1,ROUGE-2,ROUGE-L,F1 Score
0,Mistral,0.003202,0.167978,0.170492,0.000000,0.163934,0.272251
1,Phi,0.000955,0.071456,0.112821,0.007168,0.112821,0.158904
2,Qwen,0.001051,0.091359,0.120690,0.000000,0.112069,0.188586
3,Zephyr,0.002852,0.174671,0.153333,0.004329,0.146667,0.256158
